In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import numba
from numba import jit, prange
import scipy.stats as stats
import pickle
from mpl_toolkits.mplot3d import Axes3D
# @jit(nopython=True, parallel=True)

In [2]:
# Set default parameters
plt.rcParams.update({
    'figure.dpi': 150,  # Set the default DPI to 150
    'axes.grid': False,       # Display grid on axes
    'xtick.top': True,       # Display ticks on the top of the x-axis
    'xtick.bottom': True,    # Display ticks on the bottom of the x-axis
    'ytick.left': True,      # Display ticks on the left of the y-axis
    'ytick.right': True,     # Display ticks on the right of the y-axis
    'xtick.direction': 'in',  # Set the direction of x-axis ticks to 'in'
    'ytick.direction': 'in',  # Set the direction of y-axis ticks to 'in'
    'font.size': 8           # Set the font size
})

In [3]:
# # List of some commonly used rcParams settings
# rc_params = [
#     'lines.linewidth',        # Line width
#     'lines.markersize',       # Marker size
#     'axes.grid',              # Display grid on axes
#     'axes.titlesize',         # Title font size
#     'axes.labelsize',         # Axis label font size
#     'xtick.labelsize',        # X-axis tick label font size
#     'ytick.labelsize',        # Y-axis tick label font size
#     'xtick.direction',        # X-axis tick direction ('in', 'out', or 'inout')
#     'ytick.direction',        # Y-axis tick direction ('in', 'out', or 'inout')
#     'xtick.top',              # Display ticks on the top of the x-axis
#     'xtick.bottom',           # Display ticks on the bottom of the x-axis
#     'ytick.left',             # Display ticks on the left of the y-axis
#     'ytick.right',            # Display ticks on the right of the y-axis
#     'figure.figsize',         # Figure size (width, height) in inches
#     'figure.dpi',             # Dots per inch (resolution)
#     'font.size',              # Font size for text
#     'font.family',            # Font family for text
#     'legend.fontsize',        # Legend font size
#     'legend.loc',             # Legend location ('upper right', 'lower left', etc.)
# ]

In [4]:
rho054 = np.load("/vol/calvin/data/adave/Internship/SmoothDensityField054.npy")

### Code to calculate the velocity dispersion tensor

In [5]:
eigenvects054 = np.load("/vol/calvin/data/adave/Internship/eignvects_tidalshear054.npy")

In [6]:
eigenvects054.shape

(512, 512, 512, 3, 3)

In [7]:
eigenvals054 = np.load("/vol/calvin/data/adave/Internship/eignvals_tidalshear054.npy")

In [8]:
eigenvals054.shape

(512, 512, 512, 3)

In [10]:
eigenvals054[0,0,0]

array([ 0.14456867, -0.17146937, -0.71799612])

In [11]:
class_54 = np.load("/vol/calvin/data/adave/Internship/Classification_matrix054.npy")

In [12]:
class_54.shape

(512, 512, 512)

In [13]:
ids = np.load("/vol/calvin/data/adave/Internship/Particle_ids.npy")

In [14]:
ids.shape

(134217728,)

In [15]:
v_field = np.load("/vol/calvin/data/adave/Internship/Velocity_field.npy")

In [16]:
v_field.shape

(134217728, 3)

In [17]:
pos = np.load("/vol/calvin/data/adave/Internship/Particle_position.npy")

In [18]:
pos.shape

(134217728, 3)

In [19]:
box_size = 100 # h^-1 Mpc
grid_size = 512

voxel_size = box_size / grid_size

In [20]:
v_field[0]

array([-262.65454 ,  -25.674765,  111.3923  ], dtype=float32)

In [21]:
eigenvects054[0, 1, 2]

array([[-0.14504082,  0.41806231,  0.89676478],
       [ 0.54710236, -0.72129622,  0.42474788],
       [-0.82440413, -0.55222791,  0.12410548]])

In [18]:
np.dot(v_field[0], eigenvects054[0, 1, 2].T)

array([127.25467269, -77.86616464, 244.5362047 ])

In [19]:
eigenvects054[10, 10, 0]

array([[-0.47653563,  0.86793847, -0.13998789],
       [-0.02167062,  0.14758519,  0.98881191],
       [-0.87888803, -0.47423773,  0.05152083]])

In [20]:
# Store the number of particles in each voxel
voxel_particle_count = np.zeros((grid_size, grid_size, grid_size))

# Store the decomposed velocity fields
velocity_decomposition = np.zeros((grid_size**3, 3))

# Store all the particle index for respecitve voxels
voxel_info = {}

for i in tqdm(range(pos.shape[0])):   
    # Determine voxel indices for the particle
    voxel_indices = np.floor(pos[i] / voxel_size).astype(int)

    # Increment particle count in the corresponding voxel
#     voxel_particle_count[voxel_indices[0], voxel_indices[1], voxel_indices[2]] += 1
    
    # Initialize a list for the voxel if it doesn't exist in voxel_info
    voxel_key = (voxel_indices[0], voxel_indices[1], voxel_indices[2])
    if voxel_key not in voxel_info:
        voxel_info[voxel_key] = []

    # Append particle information to the list for the corresponding voxel
    voxel_info[voxel_key].append(i) # For example, particle indice xyz is in voxel (0, 0, 0)
    
    # Decompose the velocity vectors in the direction of the eigenvectors
    velocity_decomposition[i] = np.dot(v_field[i], eigenvects054[voxel_indices[0], voxel_indices[1], voxel_indices[2]].T)

100%|██████████| 134217728/134217728 [14:26<00:00, 154923.02it/s]


In [17]:
file_path = 'voxel_info_054.pkl'

# Save the dictionary to a pickle file
with open(file_path, 'wb') as pickle_file:
    pickle.dump(voxel_info, pickle_file)

In [18]:
def extract_vel_components(voxel_indices):
    # Extract all the particle indices present in a given voxel
    particle_indices = np.array(voxel_info[voxel_indices])
    
    # Determine the decomposed velocity components for all the particles
    velocities = velocity_decomposition[particle_indices][:, 0:3]
                           
    return velocities

def calculate_tensor_components(all_velocities):
    # Find the mean of the velocity components in a given voxel
    mean_velocities = np.mean(all_velocities, axis=0)
                           
    # Find the mean of the product of velocity components in a voxel
    mean_vel_products = np.mean(all_velocities[:, :, np.newaxis] * all_velocities[:, np.newaxis, :], axis=0)
    
    # Determine the dispersion tensor using the above results
    dispersion_tensor = mean_vel_products - np.outer(mean_velocities, mean_velocities)
                           
    return dispersion_tensor

In [19]:
voxel_keys = list(voxel_info.keys())

In [20]:
velocity_dispersion_tensor = np.zeros((grid_size, grid_size, grid_size, 3, 3))

for i in tqdm(range(grid_size)):
    for j in range(grid_size):
        for k in range(grid_size):
            
            voxel_indices = (i, j, k)
            
            try:
                # Find all the velocity components of all the particles in that voxel
                all_velocities = extract_vel_components(voxel_indices)
                
                # Calculate the velocity dispersion tensor using the expectation values of v_i and (v_i * v_j)
                dispersion_tensor = calculate_tensor_components(all_velocities)
                
                # Assign the calculated velocity dispersion to the corresponding voxel
                velocity_dispersion_tensor[voxel_indices + (Ellipsis,)] = dispersion_tensor
            
            except:
                pass # All the voxel indices that does not contain a particle

100%|██████████| 512/512 [14:44<00:00,  1.73s/it]


In [21]:
np.save("/vol/calvin/data/adave/Internship/Velocity_dispersion_tensor_054.npy", velocity_dispersion_tensor)

In [35]:
# Count the keys with a value containing only one element
count = sum(1 for value in voxel_info.values() if len(value) == 1)

print(f'The number of voxels containing only one particle is: {count}')


The number of voxels containing only one particle is: 16088895


### After velocity dispersion calculation

In [19]:
v_disp = np.load("/vol/calvin/data/adave/Internship/Velocity_dispersion_tensor_054.npy")

In [20]:
voxel_keys = np.load("/vol/calvin/data/adave/Internship/Voxel_keys.npy")

In [21]:
# Load the dictionary from the pickle file
with open('voxel_info_054.pkl', 'rb') as pickle_file:
    voxel_info = pickle.load(pickle_file)

In [16]:
velocity_dispersion_class = {
    'void': [],
    'sheet': [],
    'filament': [],
    'node': []
}

for voxel_indice in tqdm(voxel_keys):
    i, j, k = voxel_indice
    
    if class_54[i, j, k] == 0 and len(voxel_info[i, j, k]) > 1:
        velocity_dispersion_class['void'].append(v_disp[i, j, k])

    elif class_54[i, j, k] == 1 and len(voxel_info[i, j, k]) > 1:
        velocity_dispersion_class['sheet'].append(v_disp[i, j, k])

    elif class_54[i, j, k] == 2 and len(voxel_info[i, j, k]) > 1:
        velocity_dispersion_class['filament'].append(v_disp[i, j, k])

    elif class_54[i, j, k] == 3 and len(voxel_info[i, j, k]) > 1:
        velocity_dispersion_class['node'].append(v_disp[i, j, k])


100%|██████████| 26564815/26564815 [01:05<00:00, 403993.10it/s]


In [13]:
voxel_vals = list(voxel_info.values())

In [32]:
x = sum(len(list(v_val)) for v_val in voxel_info.values())

In [33]:
x

134217728

In [54]:
voxel_vel = np.zeros((512, 512, 512, 3))
voxel_pos = np.zeros((512, 512, 512, 3))

box_size = 100
grid_size = 512

voxel_size = box_size / grid_size

for keys in tqdm(voxel_keys):
    i, j, k = keys
    
#     all_ind = voxel_info[i, j, k]
#     mean_vox_vel = np.mean(v_field[all_ind], axis = 0)
    
#     voxel_vel[i, j, k] = mean_vox_vel
    voxel_pos[i, j, k] = [i*voxel_size, j * voxel_size, k*voxel_size]

100%|██████████| 26564815/26564815 [05:06<00:00, 86669.39it/s]


In [21]:
np.save("/vol/calvin/data/adave/Internship/voxel_mean_vel.npy", voxel_vel)

In [55]:
np.save("/vol/calvin/data/adave/Internship/voxel_mean_pos.npy", voxel_pos)

In [52]:
voxel_pos = np.load('/vol/calvin/data/adave/Internship/voxel_mean_pos.npy')

In [53]:
voxel_pos[0, 3, 0]

array([0., 0., 0.])